In [1]:
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper


from textwrap import dedent
from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()

False

In [2]:
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
PERSIST_DIRECTORY = os.getenv("PERSIST_DIRECTORY")
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_NAME")
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
MODEL_NAME = os.getenv("MODEL_NAME")
print(COLLECTION_NAME)
print(PERSIST_DIRECTORY)
print(EMBEDDING_MODEL_NAME)
print(MODEL_NAME)

ValidationError: 1 validation error for OpenAIEmbeddings
model
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

In [6]:
########################################################
# vector_db에서 데이터 불러오기
########################################################

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0 
)


retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 50,
        "fetch_k": 200,
        "lambda_mult": 0.5,
        # "filters": {"리본개수": {"$gte": 0}}
    }
)


prompt_template = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        당신은 한국의 식당을 소개하는 인공지능 비서입니다. 
        반드시 질문에 대해서 [context]에 주어진 내용을 바탕으로 답변을 해주세요. 
        질문에 '리본개수', '평점', '몇 개'라는 키워드가 포함된 경우, [context]에서 "리본개수" 항목을 확인해 답변하세요.
        리본개수는 평점과 같은 의미를 가집니다.
        [context]
        {context}
    """)),
    ("human", "{question}")
])


#########################################
# Chain 생성
#########################################

def content_from_doc(docs:list[Document]):
    return "\n\n".join([d.page_content for d in docs])


chain =  {'context': retriever  | RunnableLambda(content_from_doc), 'question': RunnablePassthrough()}  | prompt_template | model | StrOutputParser()

# QUERY 실행 및 응답 확인
QUERY = "예약 가능한 한식당 추천해줘."
response = chain.invoke(QUERY)
print("Generated Response:", response)

# 테스트 데이터 초기화
initial_test_data = [
    {"question": "예약 가능한 한식당 추천해줘.", "expected_answer": ""},
    {"question": "리본개수가 2개인 식당을 추천해주세요.", "expected_answer": ""},
    {"question": "프랑스식을 추천해주세요.", "expected_answer": ""},
    {"question": "홍보각 식당의 정보를 알려주세요", "expected_answer": ""}
]

# expected_answer 생성
def generate_expected_answers(chain, test_data):
    for test_case in test_data:
        question = test_case["question"]
        response = chain.invoke(question)
        test_case["expected_answer"] = response.strip()
    return test_data

test_data = generate_expected_answers(chain, initial_test_data)

# 성능 평가 함수
from sklearn.metrics import accuracy_score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import pandas as pd

def evaluate_chain(chain, test_data):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    results = []

    for test_case in test_data:
        question = test_case["question"]
        expected_answer = test_case["expected_answer"]

        response = chain.invoke(question)
        generated_answer = response.strip()

        correct = expected_answer in generated_answer
        bleu_score = sentence_bleu([expected_answer.split()], generated_answer.split())
        rouge_scores = scorer.score(generated_answer, expected_answer)
        precision = len(set(generated_answer.split()) & set(expected_answer.split())) / len(generated_answer.split())
        recall = len(set(generated_answer.split()) & set(expected_answer.split())) / len(expected_answer.split())
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        results.append({
            "question": question,
            "expected_answer": expected_answer,
            "generated_answer": generated_answer,
            "correct": correct,
            "bleu_score": bleu_score,
            "rouge": rouge_scores,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        })

    return results

# 평가 실행
results = evaluate_chain(chain, test_data)

# 결과 출력
for result in results:
    print(f"Question: {result['question']}")
    print(f"Expected: {result['expected_answer']}")
    print(f"Generated: {result['generated_answer']}")
    print(f"Correct: {result['correct']}")
    print(f"BLEU Score: {result['bleu_score']:.2f}")
    # ROUGE-1은 단어 단위의 겹치는 개수를 측정합니다.
    # Precision: 생성된 텍스트의 단어 중 정답 텍스트에 포함된 단어의 비율.
    # Recall: 정답 텍스트의 단어 중 생성된 텍스트에 포함된 단어의 비율.
    # F-measure: Precision과 Recall의 조화 평균.
    print(f"ROUGE: {result['rouge']}")
    print(f"Precision: {result['precision']:.2f}")
    print(f"Recall: {result['recall']:.2f}")
    print(f"F1 Score: {result['f1_score']:.2f}")
    print("-" * 30)

# 결과 저장
df_results = pd.DataFrame(results)
df_results.to_csv("evaluation_results.csv", index=False)
print("Results saved to evaluation_results.csv")


Generated Response: 예약 가능한 한식당으로는 "한식당 A"를 추천합니다. 이곳은 전통 한식을 현대적으로 재해석한 메뉴를 제공하며, 예약이 필수입니다. 또한, 분위기가 아늑하고 서비스가 뛰어나 많은 손님들에게 사랑받고 있습니다. 예약을 원하시면 미리 전화나 온라인으로 확인해 보세요.
Question: 예약 가능한 한식당 추천해줘.
Expected: 예약 가능한 한식당으로는 "한식당 A"를 추천합니다. 이곳은 전통적인 한식을 제공하며, 예약이 가능하니 미리 전화나 온라인으로 예약하시는 것이 좋습니다. 추가적인 정보가 필요하시면 말씀해 주세요!
Generated: 예약 가능한 한식당으로는 "한식당 A"를 추천합니다. 이곳은 전통 한식을 현대적으로 재해석한 메뉴를 제공하며, 예약이 필수입니다. 또한, 분위기가 아늑하고 서비스가 뛰어나 많은 손님들에게 사랑받고 있습니다. 예약을 원하시면 미리 전화나 온라인으로 확인해 보세요.
Correct: False
BLEU Score: 0.25
ROUGE: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
Precision: 0.42
Recall: 0.57
F1 Score: 0.48
------------------------------
Question: 리본개수가 2개인 식당을 추천해주세요.
Expected: 리본개수가 2개인 식당으로는 "이탈리안 레스토랑"이 있습니다. 이곳은 다양한 파스타와 피자를 제공하며, 아늑한 분위기에서 식사를 즐길 수 있습니다.
Generated: 리본개수가 2개인 식당으로는 "이탈리안 레스토랑"이 있습니다. 이곳은 다양한 파스타와 피자를 제공하며, 아늑한 분위기에서 식사를 즐길 수 있습니다. 추천 메뉴로는 트러플 크림 파스타와 마르게리타 피자가 있습니다.
Correct: True
BLEU Score: 0.66
ROUGE: {'ro

In [3]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 테스트 데이터 초기화 (expected_answer 포함)
initial_test_data = [
    {
        "question": "예약 가능한 한식당 추천해줘.",
        "expected_answer": "예약 가능한 한식당 중 추천드릴 만한 곳은 '명월관'입니다. 이곳은 워커힐호텔 내 한옥 분위기의 별채로, 주변 경치가 아름다운 숯불구이 전문점입니다. 특히 한우 오마카세를 제공하며, 최고급 소고기를 즐길 수 있습니다. 자세한 정보는 링크를 통해 확인하세요."
    },
    {
        "question": "리본개수가 2개인 식당을 추천해주세요.",
        "expected_answer": "리본 개수가 2개인 식당 중 추천드릴 만한 곳은 '메종루블랑', '더파크뷰', '블루바이필레터', '블그레', '언양기와집불고기'입니다. 이 식당들은 각각 독특한 분위기와 뛰어난 메뉴로 사랑받고 있습니다."
    },
    {
        "question": "프랑스식을 추천해주세요.",
        "expected_answer": "프랑스식 레스토랑으로는 '더그린테이블'과 '콘티넨탈'을 추천드립니다.\n\n더그린테이블: 김은희 셰프가 운영하는 프렌치 레스토랑으로, 사계절에 따라 변화하는 우리나라 식재료로 준비된 코스 요리를 제공합니다.\n콘티넨탈: 40여 년의 역사를 자랑하는 정통 유럽 스타일의 프렌치 레스토랑으로, 모던하고 세련된 분위기를 느낄 수 있습니다.\n자세한 정보를 원하시면 더그린테이블과 콘티넨탈의 웹사이트를 확인하세요."
    },
    {
        "question": "홍보각 식당의 정보를 알려주세요",
        "expected_answer": "홍보각은 국내 중식의 대가인 여경래 셰프가 운영하는 중식 파인 다이닝 레스토랑입니다. 이곳에서는 정통 중식 요리를 즐길 수 있으며, 예약이 가능합니다. 위치는 서울에 있으며, 자세한 정보는 Instagram 링크를 참고하세요."
    }
]

# 성능 평가 함수
def evaluate_chain(chain, test_data):
    # Semantic similarity 모델 로드
    embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    results = []

    for test_case in test_data:
        question = test_case["question"]
        expected_answer = test_case["expected_answer"]

        # Chain을 사용해 응답 생성
        response = chain.invoke(question)  # chain이 실제 모델과 연결되어야 함
        generated_answer = response.strip()

        # 평가 메트릭 계산
        correct = expected_answer in generated_answer
        bleu_score = sentence_bleu([expected_answer.split()], generated_answer.split())
        rouge_scores = scorer.score(generated_answer, expected_answer)
        precision = len(set(generated_answer.split()) & set(expected_answer.split())) / len(generated_answer.split())
        recall = len(set(generated_answer.split()) & set(expected_answer.split())) / len(expected_answer.split())
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        # Semantic similarity
        expected_embedding = embedding_model.encode(expected_answer)
        generated_embedding = embedding_model.encode(generated_answer)
        semantic_similarity = cosine_similarity([expected_embedding], [generated_embedding])[0][0]

        # 결과 저장
        results.append({
            "question": question,
            "expected_answer": expected_answer,
            "generated_answer": generated_answer,
            "correct": correct,
            "bleu_score": bleu_score,
            "rouge": rouge_scores,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "semantic_similarity": semantic_similarity
        })

    return results

# 평가 실행을 위한 dummy chain 객체
class DummyChain:
    def invoke(self, question):
        # 질문에 따라 응답 생성 (작성된 응답 사용)
        responses = {
            "예약 가능한 한식당 추천해줘.": "예약 가능한 한식당 중 추천드릴 만한 곳은 '명월관'입니다. 이곳은 워커힐호텔 내 한옥 분위기의 별채로, 주변 경치가 아름다운 숯불구이 전문점입니다. 특히 한우 오마카세를 제공하며, 최고급 소고기를 즐길 수 있습니다. 자세한 정보는 링크를 통해 확인하세요.",
            "리본개수가 2개인 식당을 추천해주세요.": "리본 개수가 2개인 식당 중 추천드릴 만한 곳은 '메종루블랑', '더파크뷰', '블루바이필레터', '블그레', '언양기와집불고기'입니다. 이 식당들은 각각 독특한 분위기와 뛰어난 메뉴로 사랑받고 있습니다.",
            "프랑스식을 추천해주세요.": "프랑스식 레스토랑으로는 '더그린테이블'과 '콘티넨탈'을 추천드립니다.\n\n더그린테이블: 김은희 셰프가 운영하는 프렌치 레스토랑으로, 사계절에 따라 변화하는 우리나라 식재료로 준비된 코스 요리를 제공합니다.\n콘티넨탈: 40여 년의 역사를 자랑하는 정통 유럽 스타일의 프렌치 레스토랑으로, 모던하고 세련된 분위기를 느낄 수 있습니다.\n자세한 정보를 원하시면 더그린테이블과 콘티넨탈의 웹사이트를 확인하세요.",
            "홍보각 식당의 정보를 알려주세요": "홍보각은 국내 중식의 대가인 여경래 셰프가 운영하는 중식 파인 다이닝 레스토랑입니다. 이곳에서는 정통 중식 요리를 즐길 수 있으며, 예약이 가능합니다. 위치는 서울에 있으며, 자세한 정보는 Instagram 링크를 참고하세요."
        }
        return responses.get(question, "질문에 대한 답변을 찾을 수 없습니다.")

# DummyChain 객체 생성
chain = DummyChain()

# 평가 실행
results = evaluate_chain(chain, initial_test_data)

# 결과 출력
for result in results:
    print(f"Question: {result['question']}")
    print(f"Expected: {result['expected_answer']}")
    print(f"Generated: {result['generated_answer']}")
    print(f"Correct: {result['correct']}")
    print(f"BLEU Score: {result['bleu_score']:.2f}")
    print(f"ROUGE: {result['rouge']}")
    print(f"Precision: {result['precision']:.2f}")
    print(f"Recall: {result['recall']:.2f}")
    print(f"F1 Score: {result['f1_score']:.2f}")
    print(f"Semantic Similarity: {result['semantic_similarity']:.2f}")
    print("-" * 30)

# 결과 저장
df_results = pd.DataFrame(results)
df_results.to_csv("evaluation_results.csv", index=False)
print("Results saved to evaluation_results.csv")


Question: 예약 가능한 한식당 추천해줘.
Expected: 예약 가능한 한식당 중 추천드릴 만한 곳은 '명월관'입니다. 이곳은 워커힐호텔 내 한옥 분위기의 별채로, 주변 경치가 아름다운 숯불구이 전문점입니다. 특히 한우 오마카세를 제공하며, 최고급 소고기를 즐길 수 있습니다. 자세한 정보는 링크를 통해 확인하세요.
Generated: 예약 가능한 한식당 중 추천드릴 만한 곳은 '명월관'입니다. 이곳은 워커힐호텔 내 한옥 분위기의 별채로, 주변 경치가 아름다운 숯불구이 전문점입니다. 특히 한우 오마카세를 제공하며, 최고급 소고기를 즐길 수 있습니다. 자세한 정보는 링크를 통해 확인하세요.
Correct: True
BLEU Score: 1.00
ROUGE: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0, recall=0, fmeasure=0)}
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
Semantic Similarity: 1.00
------------------------------
Question: 리본개수가 2개인 식당을 추천해주세요.
Expected: 리본 개수가 2개인 식당 중 추천드릴 만한 곳은 '메종루블랑', '더파크뷰', '블루바이필레터', '블그레', '언양기와집불고기'입니다. 이 식당들은 각각 독특한 분위기와 뛰어난 메뉴로 사랑받고 있습니다.
Generated: 리본 개수가 2개인 식당 중 추천드릴 만한 곳은 '메종루블랑', '더파크뷰', '블루바이필레터', '블그레', '언양기와집불고기'입니다. 이 식당들은 각각 독특한 분위기와 뛰어난 메뉴로 사랑받고 있습니다.
Correct: True
BLEU Score: 1.00
ROUGE: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, rec

In [5]:
import pandas as pd

data = pd.read_csv("evaluation_results.csv")
data

,question,expected_answer,generated_answer,correct,bleu_score,rouge,precision,recall,f1_score,semantic_similarity
0,예약 가능한 한식당 추천해줘.,예약 가능한 한식당 중 추천드릴 만한 곳은 '명월관'입니다. 이곳은 워커힐호텔 내 ...,예약 가능한 한식당 중 추천드릴 만한 곳은 '명월관'입니다. 이곳은 워커힐호텔 내 ...,True,1.0,"{'rouge1': Score(precision=0.0, recall=0.0, fm...",1.000000,1.000000,1.000000,1.0
1,리본개수가 2개인 식당을 추천해주세요.,"리본 개수가 2개인 식당 중 추천드릴 만한 곳은 '메종루블랑', '더파크뷰', '블...","리본 개수가 2개인 식당 중 추천드릴 만한 곳은 '메종루블랑', '더파크뷰', '블...",True,1.0,"{'rouge1': Score(precision=1.0, recall=1.0, fm...",1.000000,1.000000,1.000000,1.0
2,프랑스식을 추천해주세요.,프랑스식 레스토랑으로는 '더그린테이블'과 '콘티넨탈'을 추천드립니다.\n\n더그린테...,프랑스식 레스토랑으로는 '더그린테이블'과 '콘티넨탈'을 추천드립니다.\n\n더그린테...,True,1.0,"{'rouge1': Score(precision=1.0, recall=1.0, fm...",0.953488,0.953488,0.953488,1.0
3,홍보각 식당의 정보를 알려주세요,홍보각은 국내 중식의 대가인 여경래 셰프가 운영하는 중식 파인 다이닝 레스토랑입니다...,홍보각은 국내 중식의 대가인 여경래 셰프가 운영하는 중식 파인 다이닝 레스토랑입니다...,True,1.0,"{'rouge1': Score(precision=1.0, recall=1.0, fm...",0.928571,0.928571,0.928571,1.0
